In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pickle as pkl

In [3]:
loaded_model=pkl.load(open("/content/drive/My Drive/finalized_classifier.sav","rb"))

In [4]:
import pandas as pd
df=pd.read_csv("/content/drive/My Drive/Flight_weather.csv")
df.dropna(how='any',inplace=True)
df

,Year,Quarter,Month,DayofMonth,FlightDate,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,windspeedKmph,DewPointF,cloudcover,precipMM,pressure,WindGustKmph,visibility,weatherCode,tempF,WindChillF,winddirDegree,humidity,airport
0,2016,1,1,1,2016-01-01,741.0,0.0,745,0.0,14747,12478,1610.0,1602,0.0,8.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,SEA
1,2016,1,1,1,2016-01-01,821.0,0.0,820,1.0,14747,14107,1237.0,1204,1.0,33.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,SEA
2,2016,1,1,1,2016-01-01,759.0,1.0,724,35.0,14747,14771,1010.0,946,1.0,24.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,SEA
3,2016,1,1,1,2016-01-01,754.0,0.0,759,0.0,14747,11292,1138.0,1143,0.0,0.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,SEA
4,2016,1,1,1,2016-01-01,740.0,0.0,738,2.0,14747,12266,1401.0,1351,0.0,10.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,SEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859971,2017,4,12,17,2017-12-17,2057.0,0.0,2100,0.0,11618,13303,1.0,8,0.0,0.0,14,28,100,0.0,1024,16,10,122,32,29,123,88,EWR
1859972,2017,4,12,17,2017-12-17,2312.0,0.0,2322,0.0,13930,14771,134.0,208,0.0,0.0,10,34,100,0.0,1016,12,6,122,37,34,219,90,ORD
1859973,2017,4,12,17,2017-12-17,2306.0,1.0,2120,106.0,13930,11618,144.0,25,1.0,79.0,10,34,100,0.0,1016,12,6,122,37,34,219,90,ORD
1859974,2017,4,12,17,2017-12-17,159.0,1.0,17,102.0,14747,12266,759.0,636,1.0,83.0,10,38,100,0.8,1023,13,10,122,41,35,177,89,SEA


In [5]:
import sklearn.preprocessing as preprocessing
le=preprocessing.LabelEncoder()
df["airport"]=le.fit_transform(df["airport"].astype(str))
df["FlightDate"]=le.fit_transform(df["FlightDate"].astype(str))
df

,Year,Quarter,Month,DayofMonth,FlightDate,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,windspeedKmph,DewPointF,cloudcover,precipMM,pressure,WindGustKmph,visibility,weatherCode,tempF,WindChillF,winddirDegree,humidity,airport
0,2016,1,1,1,0,741.0,0.0,745,0.0,14747,12478,1610.0,1602,0.0,8.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,13
1,2016,1,1,1,0,821.0,0.0,820,1.0,14747,14107,1237.0,1204,1.0,33.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,13
2,2016,1,1,1,0,759.0,1.0,724,35.0,14747,14771,1010.0,946,1.0,24.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,13
3,2016,1,1,1,0,754.0,0.0,759,0.0,14747,11292,1138.0,1143,0.0,0.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,13
4,2016,1,1,1,0,740.0,0.0,738,2.0,14747,12266,1401.0,1351,0.0,10.0,7,23,0,0.0,1030,8,10,113,34,29,108,65,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859971,2017,4,12,17,716,2057.0,0.0,2100,0.0,11618,13303,1.0,8,0.0,0.0,14,28,100,0.0,1024,16,10,122,32,29,123,88,4
1859972,2017,4,12,17,716,2312.0,0.0,2322,0.0,13930,14771,134.0,208,0.0,0.0,10,34,100,0.0,1016,12,6,122,37,34,219,90,11
1859973,2017,4,12,17,716,2306.0,1.0,2120,106.0,13930,11618,144.0,25,1.0,79.0,10,34,100,0.0,1016,12,6,122,37,34,219,90,11
1859974,2017,4,12,17,716,159.0,1.0,17,102.0,14747,12266,759.0,636,1.0,83.0,10,38,100,0.8,1023,13,10,122,41,35,177,89,13


In [6]:
X=df.drop(["ArrDel15","ArrDelayMinutes","ArrTime"],axis=1)
y=df["ArrDel15"]

In [7]:
X_reg=df.drop(["ArrDel15","ArrDelayMinutes","ArrTime"],axis=1)
y_reg=df["ArrDelayMinutes"]

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
X_train_reg,X_test_reg,y_train_reg,y_test_reg=train_test_split(X_reg,y_reg,test_size=0.2,random_state=0)

In [15]:
y_pred=loaded_model.predict(X_test)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95    289814
         1.0       0.85      0.72      0.78     77020

    accuracy                           0.92    366834
   macro avg       0.89      0.84      0.86    366834
weighted avg       0.91      0.92      0.91    366834



In [41]:
delayed=pd.Series(list(map(bool,y_pred)))
delayed

0         False
1         False
2         False
3         False
4         False
          ...  
366829    False
366830     True
366831    False
366832     True
366833     True
Length: 366834, dtype: bool

In [53]:
lst=[]
for i in range(len(delayed)):
  if delayed[i]==True:
    lst.append(i)

In [87]:
X_delayed=X_test.iloc[lst]
X_delayed

,Year,Quarter,Month,DayofMonth,FlightDate,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph,DewPointF,cloudcover,precipMM,pressure,WindGustKmph,visibility,weatherCode,tempF,WindChillF,winddirDegree,humidity,airport
563888,2016,3,8,8,220,1203.0,1.0,1102,61.0,12889,12892,1220,11,41,0,0.0,1007,13,10,113,99,96,162,15,7
1292284,2017,2,5,28,513,1205.0,1.0,1119,46.0,12889,14771,1308,10,18,0,0.0,1014,20,10,113,89,91,35,7,7
164726,2016,1,3,18,77,1242.0,1.0,1202,40.0,11298,11292,1312,10,57,65,0.6,1010,12,10,176,64,64,53,79,3
1192864,2017,2,4,8,463,1510.0,1.0,1430,40.0,14771,11618,2250,27,43,100,0.3,1019,36,10,353,55,57,274,59,14
1740141,2017,4,11,5,674,1425.0,1.0,1349,36.0,12892,14747,1648,18,58,82,0.0,1018,20,10,119,61,60,245,91,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525074,2016,3,7,25,206,2028.0,1.0,1930,58.0,11298,10397,2247,19,70,100,4.3,1012,33,8,356,87,87,113,49,3
1895,2016,1,1,3,2,2005.0,1.0,1915,50.0,13303,10397,2115,9,72,62,0.0,1012,12,10,116,78,78,224,82,10
838032,2016,4,11,7,311,1701.0,1.0,1615,46.0,12266,11618,2047,10,68,99,0.5,1018,18,10,353,73,72,52,87,5
159178,2016,1,3,23,82,1505.0,1.0,1345,80.0,13204,12478,1620,15,52,33,0.0,1023,17,10,113,80,78,133,41,9


In [88]:
y_delayed=y_test_reg.iloc[lst]
y_delayed

563888     106.0
1292284     60.0
164726      34.0
1192864     18.0
1740141     49.0
           ...  
525074      40.0
1895        51.0
838032      33.0
159178      92.0
77719       41.0
Name: ArrDelayMinutes, Length: 65349, dtype: float64

In [90]:
loaded_regressor=pkl.load(open("/content/drive/My Drive/finalized_regressor.sav","rb"))

In [91]:
y_predicted_reg=loaded_regressor.predict(X_delayed)

In [92]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
print("rmse for pipeline is:",mean_squared_error(y_delayed,y_predicted_reg,squared=False))
print("MAE for pipeline is:",mean_absolute_error(y_delayed,y_predicted_reg))
print("R2_Score for pipeline is:",r2_score(y_delayed,y_predicted_reg))

rmse for pipeline is: 13.24526036915756
MAE for pipeline is: 8.790360877748704
R2_Score for pipeline is: 0.971779572344214
